# Query Optimization Homework

In this homework, we will work and play around with the Sqlite query optimizer. We will see how different join orders and indexing strategies can have a big impact on the performance.

To time how long queries take to run, we will use the built in timer feature in sqlite. This requires you to run queries in the **command line, instead of in the notebook**. You will then take the timer output and paste it back into the notebook to submit. The example below shows how to time queries in sqlite (we will use the time *real*, which is wall clock time).

```shell
[simonfrisk@vm-instunix-02] (5)$ sqlite3 3path.db
sqlite> .timer on
sqlite> SELECT COUNT(*) FROM A;
3000
Run Time: real 0.05 user 0.02 sys 0.01
```

To test that the setup is correct, you should be able to run the following.

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///TPC-H.db
Done.


name
NATION
REGION
PART
SUPPLIER
PARTSUPP
CUSTOMER
ORDERS
LINEITEM
PATH


In [25]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'    
%sql sqlite:///TPC-H.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Sqlite allows us to take a query and have the system explain which query plan it uses. Run the following query to see which plan the system uses

In [5]:
%%sql
EXPLAIN QUERY PLAN 
SELECT COUNT(*) 
FROM customer c
JOIN orders o ON c.C_CUSTKEY == o.O_CUSTKEY;

 * sqlite:///TPC-H.db
Done.


id,parent,notused,detail
4,0,0,SCAN o
6,0,0,SEARCH c USING INTEGER PRIMARY KEY (rowid=?)


Here, we can see that the query plan for the binary join works by first scanning the orders table, and then probing into the customer table for matches (all joins in Sqlite are nested-loops joins, possibly with indexes).

Furthermore, we can force Sqlite to use a specific join order, by replacing JOIN with CROSS JOIN. In the query below, we instead use the plan that scans the customer table, and probes into the order table for matches.

In [6]:
%%sql
EXPLAIN QUERY PLAN
SELECT COUNT(*) 
FROM customer c
CROSS JOIN orders o ON c.C_CUSTKEY == o.O_CUSTKEY;

 * sqlite:///TPC-H.db
Done.


id,parent,notused,detail
4,0,0,SCAN c
8,0,0,BLOOM FILTER ON o (O_CUSTKEY=?)
17,0,0,SEARCH o USING AUTOMATIC COVERING INDEX (O_CUSTKEY=?)


In this homework, we will use CROSS JOIN to manually pick the join order. This will allow us to see the performance difference of different join orders.

## 3-Path Query

We will begin by studying the following query - the 3-path query. Start by loading the following database. Then run the query below.

In [7]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'    
%sql sqlite:///3path.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
%%sql
SELECT COUNT(*)
FROM A
JOIN B ON A.b=B.b
JOIN C ON C.c=B.c;

 * sqlite:///3path.db
   sqlite:///TPC-H.db
Done.


COUNT(*)
9000000


### Question 1 (15 Points)
The above query can be computed using several different join orders. We will try the three join orders listed below.
- A joins B joins C
- A joins C joins B
- C joins B joins A

An important aspect in picking join orders is to minimize the size of the intermediate relation. If the intermediate size is large, the query might take longer time to compute. For the 3-path query above, each plan has a different intermediate relation.

For each query, do the following
- give the query in SQL using CROSS JOIN
- time the amount of time the query takes with .timer in sqlite
- give the query that computes the intermediate relation size

**A joins B joins C**

In [21]:
SELECT COUNT(*)
FROM A
CROSS JOIN B ON A.b = B.b
CROSS JOIN C ON B.c = C.c;

 * sqlite:///3path.db
   sqlite:///TPC-H.db
Done.


COUNT(*)
9000000


In [26]:
time_ABC=1.354

In [14]:
%%sql
SELECT COUNT(*)
FROM A
JOIN B ON A.b = B.b;

 * sqlite:///3path.db
   sqlite:///TPC-H.db
Done.


COUNT(*)
9000000


**A joins C joins B**

In [ ]:
SELECT COUNT(*)
FROM A
CROSS JOIN C
CROSS JOIN B
WHERE A.b = B.b AND B.c = C.c;

In [27]:
time_ACB=1.481

In [22]:
%%sql
SELECT COUNT(*)
FROM A
CROSS JOIN C;

 * sqlite:///3path.db
   sqlite:///TPC-H.db
Done.


COUNT(*)
9000000


**C joins B joins A**

In [17]:
SELECT COUNT(*)
FROM C
CROSS JOIN B ON C.c = B.c
CROSS JOIN A ON A.b = B.b;

 * sqlite:///3path.db
   sqlite:///TPC-H.db
Done.


COUNT(*)
9000000


In [28]:
time_CBA=0.199

In [19]:
%%sql
SELECT COUNT(*)
FROM C
JOIN B ON C.c = B.c;

 * sqlite:///3path.db
   sqlite:///TPC-H.db
Done.


COUNT(*)
3000


Hint: The above should show that join orders that create large intermediate results tend to be slower.

## TPC-H Query 5

In this section we play around with the TPC-H database. TPC-H consists of a database and a set of queries, and is a common benchmark to measure the performance of database systems. It consists of multiple queries that are designed to model queries appearing in business decision support systems. By timing how long time it takes for a system to execute TPC-H queries on the TPC-H database, we can compare the performance of different systems. In this question, we play with Query 5 in TPC-H, on a smaller version of the TPC-H database.

Start by loading the database

In [56]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'    
%sql sqlite:///TPC-H.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Below is Query 5, which computes the revenue in different regions of the world.

In [57]:
SELECT
    n_name, 
    sum(l_extendedprice * (1 - l_discount)) as revenue
FROM
    customer,
    orders,
    lineitem,
    supplier,
    nation,
    region
WHERE
    c_custkey = o_custkey
    AND l_orderkey = o_orderkey
    AND l_suppkey = s_suppkey
    AND c_nationkey = s_nationkey
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND o_orderdate >= '1994-01-01'
    AND o_orderdate < date('1994-01-01', '+1 year')
GROUP BY
    n_name
ORDER BY
    revenue desc;

   sqlite:///3path.db
 * sqlite:///TPC-H.db
Done.


N_NAME,revenue
VIETNAM,1000926.6999
CHINA,740210.757
JAPAN,660651.2424999999
INDONESIA,566379.5276
INDIA,422874.68439999997


Below we show the query plan that the Sqlite query optimizer picks (obtained with EXPLAIN QUERY PLAN). Note that since this is a join of 5 tables, there are many possible join orders.

QUERY PLAN  
|--SCAN lineitem  
|--SEARCH orders USING INTEGER PRIMARY KEY (rowid=?)  
|--SEARCH customer USING INTEGER PRIMARY KEY (rowid=?)  
|--SEARCH supplier USING INTEGER PRIMARY KEY (rowid=?)  
|--SEARCH nation USING INTEGER PRIMARY KEY (rowid=?)  
|--SEARCH region USING INTEGER PRIMARY KEY (rowid=?)  
|--USE TEMP B-TREE FOR GROUP BY  
`--USE TEMP B-TREE FOR ORDER BY  

### Question 4 (5 points)
For the join order picked above by sqlite
* Rewrite the query to use CROSS JOIN
* Measure how long time it takes with sqlite .timer. Paste your result

In [58]:
SELECT
    n_name, 
    sum(l_extendedprice * (1 - l_discount)) as revenue
FROM
    lineitem
CROSS JOIN orders
CROSS JOIN customer
CROSS JOIN supplier
CROSS JOIN nation
CROSS JOIN region
WHERE
    l_orderkey = o_orderkey
    AND c_custkey = o_custkey
    AND l_suppkey = s_suppkey
    AND c_nationkey = s_nationkey
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND o_orderdate >= '1994-01-01'
    AND o_orderdate < date('1994-01-01', '+1 year')
GROUP BY
    n_name
ORDER BY
    revenue desc;

   sqlite:///3path.db
 * sqlite:///TPC-H.db
Done.


N_NAME,revenue
VIETNAM,1000926.6999
CHINA,740210.757
JAPAN,660651.2424999999
INDONESIA,566379.5276
INDIA,422874.68439999997


In [29]:
time=0.033

### Question 5 (10 points)
Find another join order, where the performance is worse (the time to run is longer)
* Write the query with your picked join order using CROSS JOIN
* Measure how long time it takes with sqlite .timer. Paste your result

*Hint*: You can make it run an order of magnitude slower.

In [59]:
SELECT
    n_name, 
    sum(l_extendedprice * (1 - l_discount)) as revenue
FROM
    lineitem
CROSS JOIN customer
CROSS JOIN nation
CROSS JOIN region
CROSS JOIN orders
CROSS JOIN supplier
WHERE
    l_orderkey = o_orderkey
    AND c_custkey = o_custkey
    AND l_suppkey = s_suppkey
    AND c_nationkey = s_nationkey
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND o_orderdate >= '1994-01-01'
    AND o_orderdate < date('1994-01-01', '+1 year')
GROUP BY
    n_name
ORDER BY
    revenue desc;

   sqlite:///3path.db
 * sqlite:///TPC-H.db
Done.


N_NAME,revenue
VIETNAM,1000926.6999
CHINA,740210.757
JAPAN,660651.2424999999
INDONESIA,566379.5276
INDIA,422874.68439999997


In [60]:
time=13.633

### Question 6 (10 points)
It turns out that the join order picked by sqlite is not optimal. By using CROSS JOIN as described earlier, and picking a better join order, we can speed up the query. Find such a join order. 
* Write the query with your picked join order using CROSS JOIN
* Measure how long time it takes with sqlite .timer. Paste your result

*Hint*: The join order picked by sqlite starts by computing the join (LineItem, Orders, Customer, Supplier). This intermediate result contains tuples related to any region in the world. Those not related to the region Asia are removed only in the last join. Can we do this filtering earlier, and thereby reducing the size of the intermediate relations?

In [69]:
SELECT
    n_name, 
    SUM(l_extendedprice * (1 - l_discount)) AS revenue
FROM
    region
CROSS JOIN orders
CROSS JOIN customer
CROSS JOIN nation
CROSS JOIN lineitem
CROSS JOIN supplier
WHERE
    l_orderkey = o_orderkey
    AND c_custkey = o_custkey
    AND l_suppkey = s_suppkey
    AND c_nationkey = s_nationkey
    AND s_nationkey = n_nationkey
    AND n_regionkey = r_regionkey
    AND r_name = 'ASIA'
    AND o_orderdate >= '1994-01-01'
    AND o_orderdate < date('1994-01-01', '+1 year')
GROUP BY
    n_name
ORDER BY
    revenue desc;

   sqlite:///3path.db
 * sqlite:///TPC-H.db
Done.


N_NAME,revenue
VIETNAM,1000926.6999
CHINA,740210.757
JAPAN,660651.2424999999
INDONESIA,566379.5276
INDIA,422874.68439999997


In [31]:
time=0.012

### Question 7 (10 points)
Next, we will see if we can make the query run even faster by adding indexes. Try to add two indexes to make the query run as fast as you can.
* Write the query with your picked join order using CROSS JOIN
* Measure how long time it takes with sqlite .timer. Paste your result. It should run in a small number of milliseconds.

Make sure that if you rerun the queries above after creating the indexes, you run DROP INDEX ...

In [32]:
time=0.06